In [18]:
# ----------------------------------------------------------------------------------------
import pickle   
from os import listdir
from os.path import isfile, join
import spacy #load spacy
nlp = spacy.load('en_core_web_sm')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random as rd
from collections import defaultdict
import matplotlib.cm as cm
from copy import deepcopy

para_text = []
tfidf = []
clusters = []


def spacy_cleansing(doc):
    # for token in doc:
    #     print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
    #             token.shape_, token.is_alpha, token.is_stop)
    doc = nlp(doc)
    lemmatized = []
    for token in doc:
        if not token.is_stop and token.is_alpha:
            lemmatized.append(token.lemma_)            
    return lemmatized


def art_to_para(article,threshold=70):
    #     For fragmentation of article text into paragraphs of word-count >= threshold
    #         Parameters :
    #         -------------
    #         article : str
    #         threshold : int, optional, default 50
	para_list = article.split('\n\n')
	l = len(para_list)
	def para_thresholding(i=0, clustered_para=[]):
		temp = para_list[i]
		while len(temp.split(" ")) < threshold and i < l - 1:
			temp = temp + " " + para_list[i+1]
			i+=1
		clustered_para.append(temp) 
		if i == l-1:
			return clustered_para
		else:
			return para_thresholding(i+1,clustered_para)
	return para_thresholding()

def load_and_break(keywords='rafale deal'):
    mypath = 'news-analyzer/data/'+keywords+'/news/'
    news_text_file = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    usr_input = input("For whole article, Press a \nFor paragraph segmentation, press p\n")
    bow = [] #........cleaned documents in list of lists
    if usr_input == 'a':
        for f in news_text_file:    
            text_article = pickle.load(open(f, 'rb'))
            temp = spacy_cleansing(text_article['content'])
            bow.append(temp)
    #         print ('article no : ',news_text_file.index(f), sorted(temp),'\n')
    else:
        para_count_in_art = []
        para_list = []
        for f in news_text_file:    
            text_article = pickle.load(open(f, 'rb'))
            temp = art_to_para(text_article['content'])
            para_list += temp
            para_count_in_art.append(len(temp))
            for t in temp:
                temp1 = spacy_cleansing(t)
                bow.append(temp1)
    store_file = {}
    store_file['para_count_in_art'] = para_count_in_art if usr_input != 'a' else 0
    store_file['para_list'] = para_list if usr_input != 'a' else 0
    store_file['clean_text_doclist'] = bow
#     if selection == 'a':
#         file = 'spacy_cleansing.pkl'
#     else:
#         file = 'spacy_cleansing_of_paras.pkl'
#     pickle.dump(store_file,open(file,'wb'))
    return store_file

In [20]:
# ---------------------------------- tfidf ---------------------------------------------------------------------
processed_doc = load_and_break()

# processed_doc = pickle.load(open('spacy_cleansing_of_paras.pkl','rb'))
doc = processed_doc['clean_text_doclist']
# wordSet = sorted(set().union(*doc))

doc_set_all = []
for docc in doc:
	doc_set = sorted(set().union(docc))
	doc_dict = dict.fromkeys(doc_set, 0)
	for word in docc:
		doc_dict[word] += 1 
	doc_set_all.append(doc_dict.copy())

# print ("wordSet  :   "," ".join(sorted(wordSet)))

def computeTF(tfDict,doc):
    doc_count = len(doc)
    for word, count in tfDict.items():
    	tfDict[word] = round((count+1)/doc_count,4)
#     print(str(tfDict),'\n')
    return tfDict

tfdoc = []
for i in range(len(doc)):
    tfdoc.append(computeTF(doc_set_all[i], doc[i]))


def computeIDF(doc_set_all,wordSet):
    import math
    idfDict = dict.fromkeys(wordSet, 0)
    N = len(doc_set_all)
    for word in wordSet:
        for i in range(N):
        	if word in doc_set_all[i].keys():
        		idfDict[word] += 1
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / val)
                        
    return idfDict        


idfs = computeIDF(doc_set_all,wordSet)

def computeTFIDF(tfdoc, idfs,threshold=0.01):
    tfidf = {}
    new_wordset = [] #for thresholding
    for word, val in tfdoc.items():
        ti = round(val*idfs[word],4)
        if ti > threshold:
            tfidf[word] = ti #------rounded for error handeling of data(float64)
            new_wordset.append(word)
#     print(str(tfidf),'\n')
#     print(str(new_wordset),'\n')
    return tfidf,new_wordset

def main_tfidf(threshold=0.01):
    tfidf = []
    new_wordSet = []
    for t in tfdoc:
        temp = computeTFIDF(t, idfs,threshold)
        new_wordSet += temp[1]
        tfidf.append(temp[0])
    wordSet = sorted(set().union(new_wordSet))
    tfidf_wordSet = [dict.fromkeys(wordSet, 0)]*len(doc)
    i=0 
    tfidf_final_struc = [] 
    for wrdF in tfidf_wordSet: 
        temp = tfidf[i] 
        # print(temp, wrdF) 
        tfidf_final_struc.append(dict(wrdF, **temp)) 
        i += 1
    df = pd.DataFrame(tfidf_final_struc)
    # df.to_csv('tfidf_of_paras.csv') 
    return df, tfidf


tfidf_final = main_tfidf()
print ('SUCCESS')

For whole article, Press a 
For paragraph segmentation, press p
 P


SUCCESS


In [26]:
#------------------------------------------- k clustering 2 --------------------------------------------------------
def main_clustering():
    # https://www.kaggle.com/andyxie/k-means-clustering-implementation-in-python
    # df=pd.read_csv('tfidf_of_paras.csv')
    df = tfidf_final[0]
    X = df.iloc[:,1:].values
    X = np.nan_to_num(X)
    K=5
    m, feat = X.shape
    mean = np.mean(X, axis = 0)
    std = np.std(X, axis = 0)
    centers = np.random.randn(K,feat)*std + mean
    centers_old = np.zeros(centers.shape) # to store old centers
    centers_new = deepcopy(centers) # Store new centers
    clusters = np.zeros(m)
    distances = np.zeros((m,K))

    error = np.linalg.norm(centers_new - centers_old)

    # When, after an update, the estimate of that center stays the same, exit loop
    while error != 0:
        # Measure the distance to every center
        for i in range(K):
            distances[:,i] = np.linalg.norm(X - centers[i], axis=1)
        # Assign all training data to closest center
        clusters = np.argmin(distances, axis = 1)

        centers_old = deepcopy(centers_new)
        # Calculate mean for every cluster and update the center
        for i in range(K):
            centers_new[i] = np.mean(X[clusters == i], axis=0)
        error = np.linalg.norm(centers_new - centers_old)
    # centers_new 
    clusters
    cluster  = []
    for j in range(K):
        cluster.append(np.where(clusters == j))
    return cluster

cluster = main_clustering()
# print (cluster)
for single in cluster:
    print('==================================================================================================== \n\n\n')
    print( '\n\n\n','--------------------CLUSTER NUMBER {} WITH {}-PARA-----------:\n\n\n  '.format(cluster.index(single)+1, len(list(single[0]))))
    for i in list(single)[0]:
        print (i, '\n', processed_doc['para_list'][i], '\n\n',tfidf_final[1][i], '\n')







 --------------------CLUSTER NUMBER 1 WITH 30-PARA-----------:


  
1 
 After winning a slender majority in the 32-member legislative assembly, a Sikkim Krantikari Morcha (SKM) delegation led by its president P S Golay on Saturday called on Governor Ganga Prasad and staked claim to form the next government in the state. The SKM leaders, however, were tight-lipped about the name of their legislature party chief. 

 {'Ganga': 0.1432, 'Golay': 0.1432, 'Governor': 0.1432, 'Krantikari': 0.173, 'Morcha': 0.173, 'P': 0.1294, 'Prasad': 0.1166, 'S': 0.1244, 'SKM': 0.2149, 'Saturday': 0.1294, 'Sikkim': 0.1432, 'assembly': 0.1134, 'call': 0.0891, 'chief': 0.0758, 'claim': 0.0518, 'delegation': 0.1542, 'form': 0.0978, 'government': 0.0237, 'lead': 0.0758, 'leader': 0.0703, 'legislative': 0.1294, 'legislature': 0.1542, 'lipped': 0.173, 'majority': 0.1294, 'party': 0.0558, 'president': 0.0628, 'slend': 0.173, 'stake': 0.1294, 'state': 0.0658, 'tight': 0.173, 'win': 0.1166} 

27 
 "I challenge 

/home/gayar/.virtualenvs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


 As much as the BJP is trying to reject allegations of corruption in the Rafale deal, the Congress party is not going to let go of Hollande's statement any time soon. Mediapart, a French language publication, quoted Hollande as saying, "It was the Indian government that proposed this service group (Reliance Defence), and Dassault who negotiated with Ambani. We had no choice, we took the interlocutor who was given to us." Asked who selected Reliance as a partner and why, Hollande replied, "We had no say in this regard." 

 {'Ambani': 0.0376, 'BJP': 0.0451, 'Congress': 0.022, 'Dassault': 0.0348, 'Defence': 0.0344, 'Hollande': 0.1095, 'Mediapart': 0.0844, 'Rafale': 0.0109, 'Reliance': 0.0608, 'allegation': 0.0553, 'ask': 0.0451, 'choice': 0.0673, 'corruption': 0.0558, 'deal': 0.0128, 'french': 0.0349, 'give': 0.0461, 'go': 0.0569, 'government': 0.0176, 'group': 0.0822, 'indian': 0.0471, 'interlocutor': 0.1007, 'language': 0.1147, 'let': 0.0894, 'negotiate': 0.0542, 'partner': 0.0506, 'par